### Transformação dos dados

Nesse notebook vamos transformar os dados para identificar o momento (ano) em que uma data escola entrou no PEI.

In [1]:
import pandas as pd

from utils.load_csv import load_csv


In [2]:
df = load_csv('dados_merged.csv')

In [3]:
df.head()

,ano_censo,sigla_uf,nome_municipio,codigo_municipio,codigo_escola,nome_escola,qtd_turmas_ed_basica,qtd_matr_ed_basica,qtd_matr_ed_basica_nao_declarada,qtd_matr_ed_basica_branca,...,qtd_matriculas_infantil_integral,qtd_matriculas_infantil_creche_integral,qtd_matriculas_infantil_pre_escolar_integral,qtd_matriculas_fundamental_integral,qtd_matriculas_fundamental_anos_iniciais_integral,qtd_matriculas_fundamental_anos_finais_integral,qtd_matriculas_medio_integral,coordenadas,is_PEI,_merge
0,2011,SP,adamantina,3500105,35030806,helen keller,25,830,133,455,...,0,0,0,303,0,303,0,"['-21.693', '-51.0689']",True,both
1,2011,SP,adamantina,3500105,35031045,durvalino grion prof,22,771,94,446,...,0,0,0,0,0,0,0,"['-21.6786', '-51.0769']",True,both
2,2011,SP,adamantina,3500105,35031082,eudecio luiz vicente prof ete,22,762,111,498,...,0,0,0,0,0,0,0,NaN,False,left_only
3,2011,SP,adamantina,3500105,35031100,herval bellusci engenheiro ete,10,267,73,135,...,0,0,0,0,0,0,78,NaN,False,left_only
4,2011,SP,adamantina,3500105,35031112,fleurides cavallini menechino profa,32,1102,61,829,...,0,0,0,0,0,0,1,"['-21.6928', '-51.0703']",True,both


In [4]:
df.columns

Index(['ano_censo', 'sigla_uf', 'nome_municipio', 'codigo_municipio',
       'codigo_escola', 'nome_escola', 'qtd_turmas_ed_basica',
       'qtd_matr_ed_basica', 'qtd_matr_ed_basica_nao_declarada',
       'qtd_matr_ed_basica_branca', 'qtd_matr_ed_basica_preta',
       'qtd_matr_ed_basica_parda', 'qtd_matr_ed_basica_amarela',
       'qtd_matr_ed_basica_indigena', 'qtd_matriculas_infantil_integral',
       'qtd_matriculas_infantil_creche_integral',
       'qtd_matriculas_infantil_pre_escolar_integral',
       'qtd_matriculas_fundamental_integral',
       'qtd_matriculas_fundamental_anos_iniciais_integral',
       'qtd_matriculas_fundamental_anos_finais_integral',
       'qtd_matriculas_medio_integral', 'coordenadas', 'is_PEI', '_merge'],
      dtype='object')

Checagem: todos os dados são do estado de São Paulo

In [5]:
df['sigla_uf'].unique()

array(['SP'], dtype=object)

Checagem: os anos são os do período de estudo

In [6]:
df['ano_censo'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [7]:
df['_merge'].value_counts()

_merge
left_only    28560
both         18287
Name: count, dtype: int64

Checagem: a quantidade de escolas marcadas como PEI tem que ser igual ao merge que deu both

In [8]:
df['is_PEI'].sum() == (df['_merge']=='both').sum()

np.True_

In [9]:
matriculas_integral = [col for col in df.columns if col.endswith('_integral')]
matriculas_integral

['qtd_matriculas_infantil_integral',
 'qtd_matriculas_infantil_creche_integral',
 'qtd_matriculas_infantil_pre_escolar_integral',
 'qtd_matriculas_fundamental_integral',
 'qtd_matriculas_fundamental_anos_iniciais_integral',
 'qtd_matriculas_fundamental_anos_finais_integral',
 'qtd_matriculas_medio_integral']

In [18]:
df[matriculas_integral].max()

qtd_matriculas_infantil_integral                      258
qtd_matriculas_infantil_creche_integral               141
qtd_matriculas_infantil_pre_escolar_integral          132
qtd_matriculas_fundamental_integral                   774
qtd_matriculas_fundamental_anos_iniciais_integral     532
qtd_matriculas_fundamental_anos_finais_integral       774
qtd_matriculas_medio_integral                        1111
dtype: int64

In [10]:
df['total_matriculas_basica_integral'] = df[matriculas_integral].sum(axis=1)

In [11]:
df['possui_integral'] = df['total_matriculas_basica_integral'] > 0

In [12]:
df['possui_integral'].sum()

np.int64(12965)

In [14]:
df['is_PEI'].sum()

np.int64(18287)

### Quantidade de escolas marcadas como PEI versus quantidade de escolas com matrículas no integral

Como esperado, temos mais escolas marcadas como PEI do que escolas que possuem efetivamente matrículas no integral.

Isso ocorre porque o nosso dado do PEI é de 2025. E o estudo para em 2018. Dessa forma, temos escolas que entraram no integral posteriormente ao período de estudo e que não possuem matrículas no integral na base do censo até 2018. Então isso está como o esperado.

Além disso, há escolas que possuem matrículas no integral para anos que não deveriam ter segundo a política do PEI que estamos estudando agora, que se restringe ao FUND II e Médio. Então vamos fazer uma nova coluna que identifica a quantidades de matrícula no integral para o PEI.



In [15]:
df['is_PEI'].sum() - df['possui_integral'].sum()

np.int64(5322)

In [20]:
matriculas_integral_pei = ['qtd_matriculas_fundamental_anos_finais_integral', 'qtd_matriculas_medio_integral']

In [21]:
df['total_matriculas_integral_PEI'] = df[matriculas_integral_pei].sum(axis=1)

In [22]:
df['possui_integral_pei'] = df['total_matriculas_integral_PEI'] > 0
df['possui_integral_pei'].sum()

np.int64(10255)

In [23]:
df['is_PEI'].sum() -  df['possui_integral_pei'].sum()

np.int64(8032)

In [24]:
df['is_pei_and_possui_integral_pei'] = df['is_PEI'] & df['possui_integral_pei']
df['is_pei_and_possui_integral_pei'].sum()

np.int64(5459)

#### Provavelmente as escolas abaixo são as que entraram no PEI posteriormente

In [25]:
df['is_pei_and_not_possui_integral_pei'] = df['is_PEI'] & ~df['possui_integral_pei']
df['is_pei_and_not_possui_integral_pei'].sum()

np.int64(12828)

Escolas que possuem integral no PEI mas não são PEI (temos que entender essas)

In [26]:
df['possui_integral_pei_and_not_pei'] = df['possui_integral_pei'] & ~df['is_PEI']
df['possui_integral_pei_and_not_pei'].sum()

np.int64(4796)

#### A inspeção mostra que são escolas de ensino do tipo ETEC.

Vamos voltar ao censo para filtrar isso.

In [27]:
df[df['possui_integral_pei_and_not_pei']]

,ano_censo,sigla_uf,nome_municipio,codigo_municipio,codigo_escola,nome_escola,qtd_turmas_ed_basica,qtd_matr_ed_basica,qtd_matr_ed_basica_nao_declarada,qtd_matr_ed_basica_branca,...,coordenadas,is_PEI,_merge,total_matriculas_basica_integral,possui_integral,total_matriculas_integral_PEI,possui_integral_pei,is_pei_and_possui_integral_pei,is_pei_and_not_possui_integral_pei,possui_integral_pei_and_not_pei
3,2011,SP,adamantina,3500105,35031100,herval bellusci engenheiro ete,10,267,73,135,...,NaN,False,left_only,78,True,78,True,False,False,True
22,2011,SP,agudos,3500709,35049670,farid fayad prof,20,676,167,383,...,NaN,False,left_only,4,True,2,True,False,False,True
61,2011,SP,americana,3501608,35045962,polivalente de americana ete,82,2782,665,1726,...,NaN,False,left_only,38,True,38,True,False,False,True
85,2011,SP,amparo,3501905,35017565,nelson alves de godoy dr,19,532,46,321,...,NaN,False,left_only,10,True,5,True,False,False,True
88,2011,SP,amparo,3501905,35047041,dionysia gerbi beira,24,765,49,494,...,NaN,False,left_only,2,True,1,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46808,2018,SP,vera cruz,3556602,35033595,paulo guerreiro franco etec,9,254,20,146,...,NaN,False,left_only,197,True,197,True,False,False,True
46813,2018,SP,viradouro,3556800,35022585,odulfo de oliveira guimaraes,24,738,24,469,...,NaN,False,left_only,106,True,64,True,False,False,True
46825,2018,SP,votorantim,3557006,35405218,elias miguel junior professor etec,20,683,118,474,...,NaN,False,left_only,271,True,271,True,False,False,True
46835,2018,SP,votuporanga,3557105,35029087,arnaldo maria de itaporanga frei etec,20,673,81,453,...,NaN,False,left_only,422,True,422,True,False,False,True


In [28]:
df_censo = load_csv('microdados_censo_limpo.csv')

In [29]:
df_censo

,ano_censo,sigla_uf,nome_municipio,codigo_municipio,codigo_escola,nome_escola,qtd_turmas_ed_basica,qtd_matr_ed_basica,qtd_matr_ed_basica_nao_declarada,qtd_matr_ed_basica_branca,...,qtd_matr_ed_basica_parda,qtd_matr_ed_basica_amarela,qtd_matr_ed_basica_indigena,qtd_matriculas_infantil_integral,qtd_matriculas_infantil_creche_integral,qtd_matriculas_infantil_pre_escolar_integral,qtd_matriculas_fundamental_integral,qtd_matriculas_fundamental_anos_iniciais_integral,qtd_matriculas_fundamental_anos_finais_integral,qtd_matriculas_medio_integral
0,2011,SP,Adamantina,3500105,35030806,HELEN KELLER,25,830,133,455,...,225,2,2,0,0,0,303,0,303,0
1,2011,SP,Adamantina,3500105,35031045,DURVALINO GRION PROF,22,771,94,446,...,210,6,2,0,0,0,0,0,0,0
2,2011,SP,Adamantina,3500105,35031082,EUDECIO LUIZ VICENTE PROF ETE,22,762,111,498,...,133,18,1,0,0,0,0,0,0,0
3,2011,SP,Adamantina,3500105,35031100,HERVAL BELLUSCI ENGENHEIRO ETE,10,267,73,135,...,54,1,0,0,0,0,0,0,0,78
4,2011,SP,Adamantina,3500105,35031112,FLEURIDES CAVALLINI MENECHINO PROFA,32,1102,61,829,...,193,11,3,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46842,2018,SP,Votuporanga,3557105,35919697,MARIA NIVEA COSTA PINTO FREITAS PROFA,14,376,26,313,...,29,2,0,0,0,0,0,0,0,0
46843,2018,SP,Votuporanga,3557105,35985739,CEL JTO A EE JOSE MANOEL LOBO DR,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46844,2018,SP,Zacarias,3557154,35027418,ANTONIO TEIXEIRA DOS SANTOS,8,210,7,164,...,35,3,0,0,0,0,0,0,0,0
46845,2018,SP,Chavantes,3557204,35033893,ERNESTO FONSECA DOUTOR,31,921,56,596,...,241,1,1,0,0,0,0,0,0,0
